### Import dependencies

In [190]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import SGD

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.metrics import confusion_matrix
import time
import numpy as np

In [191]:
print(torch.cuda.is_available())

False


In [192]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Coding basic NN

In [193]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class BasicPerceptron(nn.Module):
    def __init__(self, input_size, num_classes):
        super(BasicPerceptron, self).__init__()
        self.weights = nn.Parameter(torch.randn(num_classes, input_size), requires_grad=True)
        self.biases = nn.Parameter(torch.randn(num_classes), requires_grad=True)
        self.num_classes = num_classes

    def forward(self, x):
        # Multiply input by weights and add biases
        linear_output = torch.matmul(x, self.weights.t()) + self.biases
        return linear_output

    def train_model(self, input_data, target_labels, num_epochs=1000, learning_rate=0.01):
        # Define the loss function
        criterion = nn.CrossEntropyLoss()
        # Define the optimizer
        optimizer = optim.SGD(self.parameters(), lr=learning_rate)

        for epoch in range(num_epochs):
            # Forward pass
            outputs = self(input_data)

            # Compute the loss
            loss = criterion(outputs, target_labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Print the loss every few epochs
            if epoch % 10 == 0:
                print(f"Epoch {epoch}: Loss {loss.item()}")

    def predict(self, input_data):
        # Forward pass to get the output logits
        logits = self(input_data)
        # Apply softmax to get probabilities
        probabilities = F.softmax(logits, dim=1)
        # Get the class label with highest probability
        predicted_labels = torch.argmax(probabilities, dim=1)
        return predicted_labels


    def estimate(self, y_true, y_pred):
      """
      calculates the unweighted average recall across all the classes present
      in the true labels (y_true)
      and predicted labels (y_pred) using the confusion matrix.
      """
      cm = confusion_matrix(y_true, y_pred)
      num_classes = len(cm)
      recalls = []
      for i in range(self.num_classes):
          true_positives = cm[i, i]
          actual_positives = sum(cm[i, :])
          recall = true_positives / actual_positives
          recalls.append(recall)
      average_recall = sum(recalls) / num_classes
      return average_recall

### Experiment

In [194]:
# Load data from CSV files using pandas
features_df = pd.read_csv('/content/feature_mfcc_34_delta_delta_nfft_8192.csv', index_col=0)
labels_df = pd.read_csv('/content/y_labels_feature_34_mfcc_delta_delta_nfft_8192.csv', index_col=0)

# Convert features and labels to numpy arrays
features = features_df.values
labels_df.replace({'neutral':0, 'calm':1, 'happy':2, 'sad':3, 'angry':4, 'fear':5, 'disgust':6, 'surprised':7}, inplace=True)
labels = labels_df.values
print(labels)

# Convert numpy arrays to PyTorch tensors
x_tensor = torch.tensor(features, dtype=torch.float32)
y_tensor = torch.tensor(labels, dtype=torch.long).squeeze()  # Assuming labels are integers (long tensor)

# Print shape of tensors
print("Shape of x_tensor:", x_tensor.shape)
print("Shape of y_tensor:", y_tensor.shape)

[[7]
 [2]
 [1]
 ...
 [7]
 [6]
 [5]]
Shape of x_tensor: torch.Size([1440, 306])
Shape of y_tensor: torch.Size([1440])


In [201]:
train_ratio = 0.7  # 80% of the data for training, 20% for testing
# Calculate the number of samples for training and testing
train_size = int(train_ratio * len(x_tensor))
test_size = len(x_tensor) - train_size

# Shuffle the indices of the samples
indices = torch.randperm(len(x_tensor))

# Split the indices into training and testing sets
train_indices = indices[:train_size]
test_indices = indices[train_size:]

# Use the indices to extract the samples for training and testing
x_train, y_train = x_tensor[train_indices], y_tensor[train_indices]
x_test, y_test = x_tensor[test_indices], y_tensor[test_indices]

# Print the shapes of the training and testing sets
print("Shape of x_train:", x_train.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of x_test:", x_test.shape)
print("Shape of y_test:", y_test.shape)

Shape of x_train: torch.Size([1007, 306])
Shape of y_train: torch.Size([1007])
Shape of x_test: torch.Size([433, 306])
Shape of y_test: torch.Size([433])


In [204]:
model = BasicPerceptron(input_size=306, num_classes=8)
model.train_model(x_train, y_train, num_epochs=5000, learning_rate=0.1)

# 2. Predict labels for the testing/validation data
y_pred = model.predict(x_test)

# 3. Calculate accuracy
uar = model.estimate(y_test, y_pred)

print(f"UAR: {uar}")

Epoch 0: Loss 22.201038360595703
Epoch 10: Loss 12.007477760314941
Epoch 20: Loss 9.196626663208008
Epoch 30: Loss 7.431214332580566
Epoch 40: Loss 6.15256404876709
Epoch 50: Loss 5.200140476226807
Epoch 60: Loss 4.501382350921631
Epoch 70: Loss 3.9571022987365723
Epoch 80: Loss 3.51375412940979
Epoch 90: Loss 3.1484124660491943
Epoch 100: Loss 2.842606544494629
Epoch 110: Loss 2.582691192626953
Epoch 120: Loss 2.359424114227295
Epoch 130: Loss 2.167097806930542
Epoch 140: Loss 2.000316619873047
Epoch 150: Loss 1.853821039199829
Epoch 160: Loss 1.7234177589416504
Epoch 170: Loss 1.6062692403793335
Epoch 180: Loss 1.5004349946975708
Epoch 190: Loss 1.4046496152877808
Epoch 200: Loss 1.3181558847427368
Epoch 210: Loss 1.2400637865066528
Epoch 220: Loss 1.169301152229309
Epoch 230: Loss 1.1049368381500244
Epoch 240: Loss 1.046176552772522
Epoch 250: Loss 0.9923300743103027
Epoch 260: Loss 0.9428204298019409
Epoch 270: Loss 0.8972203135490417
Epoch 280: Loss 0.8552097082138062
Epoch 290: L

### TRY K-FOLD

In [176]:
def load_dataset(X_path='/content/feature_mfcc_34_delta_delta_nfft_8192.csv',
                 y_path='/content/y_labels_feature_34_mfcc_delta_delta_nfft_8192.csv',
                 ID_path='/content/IDs_feature_mfcc_34_delta_delta_nfft_8192.csv'):
    starting_time = time.time()
    X = pd.read_csv(X_path)
    X = X.drop('Unnamed: 0', axis=1)
    y = pd.read_csv(y_path)
    y = y.drop('Unnamed: 0', axis=1)
    ID = pd.read_csv(ID_path)
    ID = ID.drop('Unnamed: 0', axis=1)

    print("data loaded in " + str(time.time() - starting_time) + "ms")
    print(X.head())
    print("X.shape = ", X.shape)
    print("X.columns = ", X.columns)

    return X, y, ID

def get_k_fold_group_member():
    return {
        '0': {2, 5, 14, 15, 16},
        '1': {3, 6, 7, 13, 18},
        '2': {10, 11, 12, 19, 20},
        '3': {8, 17, 21, 23, 24},
        '4': {1, 4, 9, 22}
    }

def get_custom_k_folds(X, y, ID, group_members):
    X_k_fold = dict()
    y_k_fold = dict()
    for k, members in group_members.items():
        fold_X = pd.DataFrame()
        fold_y = pd.DataFrame()

        for actor_ID in members:
            inds = ID[ID['0'] == actor_ID].index.tolist()
            fold_X = pd.concat([fold_X, X.loc[inds, :]])
            fold_y = pd.concat([fold_y, y.loc[inds, :]])
        X_k_fold[k] = fold_X
        y_k_fold[k] = fold_y
    return X_k_fold, y_k_fold

In [177]:
# Load data from CSV files using pandas
X = pd.read_csv('/content/feature_mfcc_34_delta_delta_nfft_8192.csv', index_col=0)
y = pd.read_csv('/content/y_labels_feature_34_mfcc_delta_delta_nfft_8192.csv', index_col=0)
IDs = pd.read_csv('/content/IDs_feature_mfcc_34_delta_delta_nfft_8192.csv', index_col=0)

group_members= get_k_fold_group_member()


X_k_folds, y_k_folds = get_custom_k_folds(X, y, IDs, group_members)

In [178]:
print(y_k_folds)

{'0':               0
1380       calm
1381       calm
1382    neutral
1383      angry
1384  surprised
...         ...
595     disgust
596         sad
597         sad
598       angry
599   surprised

[300 rows x 1 columns], '1':              0
120       calm
121      angry
122      angry
123        sad
124  surprised
..         ...
955    disgust
956       calm
957      angry
958        sad
959        sad

[300 rows x 1 columns], '2':               0
1200      happy
1201        sad
1202    neutral
1203    neutral
1204    neutral
...         ...
475   surprised
476       angry
477       happy
478   surprised
479        calm

[300 rows x 1 columns], '3':               0
1320        sad
1321       calm
1322    disgust
1323  surprised
1324       calm
...         ...
295   surprised
296   surprised
297         sad
298       angry
299     neutral

[300 rows x 1 columns], '4':               0
1260      happy
1261    disgust
1262    disgust
1263    disgust
1264       fear
...         ...
55    

In [181]:
def estimate_model(model, X_k_folds, y_k_folds, num_epochs=10000):
    y_pred = []
    y_true = []

    for k in X_k_folds.keys():
        # Prepare dataset
        X_train = pd.DataFrame()
        y_train = pd.DataFrame()
        for i in X_k_folds.keys():
            if (i != k):
                X_train = pd.concat([X_train, X_k_folds[i]])
                y_train = pd.concat([y_train, y_k_folds[i]])
            else:
                X_test = X_k_folds[i]
                y_test = y_k_folds[i]
        x_tensor = torch.tensor(X_train.values, dtype=torch.float32)
        y_train_normalized = pd.Series(y_train.values.ravel()).replace({'neutral':0, 'calm':1, 'happy':2, 'sad':3, 'angry':4, 'fear':5, 'disgust':6, 'surprised':7})
        y_tensor = torch.tensor(y_train_normalized)  # Assuming labels are integers (long tensor)
        model.train_model(x_tensor, y_tensor)
        y_pred_k_fold = model.predict(torch.tensor(X_test.values, dtype=torch.float32))

        y_pred = np.concatenate((y_pred, y_pred_k_fold), axis=None)
        y_true = np.concatenate((y_true, y_test.values), axis=None)

    UAR = model.estimate(y_true, y_pred)
    # print(f'UAR = {UAR:.3f}')
    return UAR, y_pred, y_true

In [198]:
model = BasicPerceptron(input_size=306, num_classes=8)
uar, y_pred, y_true=estimate_model(model, X_k_folds, y_k_folds, num_epochs=10)

print(f"UAR: {uar}")